<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Paquet-interpretation" data-toc-modified-id="Paquet-interpretation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Paquet interpretation</a></span><ul class="toc-item"><li><span><a href="#Decompose-the-paquet" data-toc-modified-id="Decompose-the-paquet-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Decompose the paquet</a></span></li><li><span><a href="#Mapping-the-Ids" data-toc-modified-id="Mapping-the-Ids-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Mapping the Ids</a></span><ul class="toc-item"><li><span><a href="#Gotta-need-to-decompile-the-source-code-first" data-toc-modified-id="Gotta-need-to-decompile-the-source-code-first-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Gotta need to decompile the source code first</a></span></li><li><span><a href="#Then-extract-the-mapping-of-the-Ids" data-toc-modified-id="Then-extract-the-mapping-of-the-Ids-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Then extract the mapping of the Ids</a></span></li></ul></li></ul></li><li><span><a href="#Deserialize" data-toc-modified-id="Deserialize-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Deserialize</a></span></li></ul></div>

In [1]:
import pyshark
import nest_asyncio
nest_asyncio.apply()

In [2]:
capture = pyshark.LiveCapture(interface='en0', bpf_filter='tcp port 5555 and len > 66')

KeyboardInterrupt: 

In [3]:
capture.sniff(timeout=20)

In [4]:
print(capture)

<LiveCapture (27 packets)>


In [ ]:
6999

In [7]:
\print("[DEBUG] {} paquets reçus".format( len(capture) ) )
for packet in capture:
            print(packet.tcp.payload)

[DEBUG] 23 paquets reçus
54:e1:bd:14:08:42:54:66:8d:80:34:80:00:12:1f:00:ee:01:01:00:06:5a:df:10:ef:01:b3:0b:43:ce:04:00:09:01:ff:c8:67:02:f4:76:00:03:4e:4e:4e:04:6a:00:00:05:fa:97:17:07:ff:ff:ff:08:0d:04:ee:09:4e:a1:fe:0a:00:00:00:00:01:8c:01:00:00:00:0f:50:69:78:65:6c:2d:4c:65:2d:52:65:74:6f:75:72:0d:e0:00:08:00:00:00:04:25:6f:ca:95:01:00:0b:41:2d:53:2d:54:2d:52:2d:4f:2d:53:26:0f:00:0d:04:ee:04:00:ff:ff:ff:13:29:8a:16:00:04:43:52:4f:43:00:10:43:72:6f:71:75:61:6e:74:20:63:72:6f:71:75:65:72:ce:03:00:00:00:00:03:00:4e:a1:fe:00:14:a5:45:00:00:1d:62:01:0a:1e:bb:de:00:00:00:42:54:66:8d:80:3c:00:00
8d:b1:0b:00:01:d2:81:d8:d1:99:0a:00:01:00
1f:31:0b:42:54:66:8d:80:34:80:00:00:08:00


KeyboardInterrupt: 

In [8]:
len(capture)

23

In [150]:
packet_test = capture[1]

In [151]:
packet_test.DATA.data

'87810316ee0852311000020125014100014253cfb300348000345c'

In [152]:
my_packet = packet_test.tcp.payload
my_packet

'87:81:03:16:ee:08:52:31:10:00:02:01:25:01:41:00:01:42:53:cf:b3:00:34:80:00:34:5c'

# Paquet interpretation

## Decompose the paquet

In [1]:
class Packet:
    def __init__(self, packetPayLoad):
        self.payload = packetPayLoad
        self.header = self.get_header()
        self.id = self.get_id()
        self.size_of_size = self.get_size_of_size()
        self.size_packet = self.get_size_packet()
        self.size_message = self.get_size_message()
        
    def get_header(self):
        return ''.join(self.payload.split(':')[:2])
        
    def get_bin_header(self):
        scale = 16 #hexadecimal
        num_of_bits = 16
        header = self.header
        
        return bin(int(header, scale))[2:].zfill(num_of_bits)
        
    def get_id(self):
        if ~hasattr(self, "bin_header"):
            self.bin_header = self.get_bin_header()
        
        return int(self.bin_header[:14], 2)
    
    def get_size_of_size(self):
        if ~hasattr(self, "bin_header"):
            self.bin_header = self.get_bin_header()
        return int(self.bin_header[-2:], 2)
    
    def get_size_packet(self):
        return ''.join(self.payload.split(":")[2 : 2+self.size_of_size])
    
    def get_size_message(self):
        return int(self.size_packet, base=16)

In [6]:
my_packet = "54:e1:bd:14:08:42:54:66:8d:80:34:80:00:12:1f:00:ee:01:01:00:06:5a:df:10:ef:01:b3:0b:43:ce:04:00:09:01:ff:c8:67:02:f4:76:00:03:4e:4e:4e:04:6a:00:00:05:fa:97:17:07:ff:ff:ff:08:0d:04:ee:09:4e:a1:fe:0a:00:00:00:00:01:8c:01:00:00:00:0f:50:69:78:65:6c:2d:4c:65:2d:52:65:74:6f:75:72:0d:e0:00:08:00:00:00:04:25:6f:ca:95:01:00:0b:41:2d:53:2d:54:2d:52:2d:4f:2d:53:26:0f:00:0d:04:ee:04:00:ff:ff:ff:13:29:8a:16:00:04:43:52:4f:43:00:10:43:72:6f:71:75:61:6e:74:20:63:72:6f:71:75:65:72:ce:03:00:00:00:00:03:00:4e:a1:fe:00:14:a5:45:00:00:1d:62:01:0a:1e:bb:de:00:00:00:42:54:66:8d:80:3c:00:00:8d:b1:0b:00:01:d2:81:d8:d1:99:0a:00:01:00:1f:31:0b:42:54:66:8d:80:34:80:00:00:08:00"
# capture[1].tcp.payload
print(my_packet)

54:e1:bd:14:08:42:54:66:8d:80:34:80:00:12:1f:00:ee:01:01:00:06:5a:df:10:ef:01:b3:0b:43:ce:04:00:09:01:ff:c8:67:02:f4:76:00:03:4e:4e:4e:04:6a:00:00:05:fa:97:17:07:ff:ff:ff:08:0d:04:ee:09:4e:a1:fe:0a:00:00:00:00:01:8c:01:00:00:00:0f:50:69:78:65:6c:2d:4c:65:2d:52:65:74:6f:75:72:0d:e0:00:08:00:00:00:04:25:6f:ca:95:01:00:0b:41:2d:53:2d:54:2d:52:2d:4f:2d:53:26:0f:00:0d:04:ee:04:00:ff:ff:ff:13:29:8a:16:00:04:43:52:4f:43:00:10:43:72:6f:71:75:61:6e:74:20:63:72:6f:71:75:65:72:ce:03:00:00:00:00:03:00:4e:a1:fe:00:14:a5:45:00:00:1d:62:01:0a:1e:bb:de:00:00:00:42:54:66:8d:80:3c:00:00:8d:b1:0b:00:01:d2:81:d8:d1:99:0a:00:01:00:1f:31:0b:42:54:66:8d:80:34:80:00:00:08:00


In [7]:
testing_a_packet_class = Packet(my_packet)

In [8]:
testing_a_packet_class.payload

'54:e1:bd:14:08:42:54:66:8d:80:34:80:00:12:1f:00:ee:01:01:00:06:5a:df:10:ef:01:b3:0b:43:ce:04:00:09:01:ff:c8:67:02:f4:76:00:03:4e:4e:4e:04:6a:00:00:05:fa:97:17:07:ff:ff:ff:08:0d:04:ee:09:4e:a1:fe:0a:00:00:00:00:01:8c:01:00:00:00:0f:50:69:78:65:6c:2d:4c:65:2d:52:65:74:6f:75:72:0d:e0:00:08:00:00:00:04:25:6f:ca:95:01:00:0b:41:2d:53:2d:54:2d:52:2d:4f:2d:53:26:0f:00:0d:04:ee:04:00:ff:ff:ff:13:29:8a:16:00:04:43:52:4f:43:00:10:43:72:6f:71:75:61:6e:74:20:63:72:6f:71:75:65:72:ce:03:00:00:00:00:03:00:4e:a1:fe:00:14:a5:45:00:00:1d:62:01:0a:1e:bb:de:00:00:00:42:54:66:8d:80:3c:00:00:8d:b1:0b:00:01:d2:81:d8:d1:99:0a:00:01:00:1f:31:0b:42:54:66:8d:80:34:80:00:00:08:00'

In [9]:
print("The header hexadecimal value is %s" %(testing_a_packet_class.header))
print("The packet ID is %s while the size of the size is %s" %(testing_a_packet_class.id,
                                                             testing_a_packet_class.size_of_size))
print("The message size in hexa is then %s which represents %s bytes after conversion" 
      %(testing_a_packet_class.size_packet, testing_a_packet_class.size_message))

The header hexadecimal value is 54e1
The packet ID is 5432 while the size of the size is 1
The message size in hexa is then bd which represents 189 bytes after conversion


## Mapping the Ids

### Gotta need to decompile the source code first

USE JPEXS Free Flash Decompiler to decompile the source code, see decompile.sh

### Then extract the mapping of the Ids

In [94]:
import os

def Ids(path='/Users/thomasaudevie/Desktop/projets-perso/kamarketplace/sources/scripts/com/ankamagames/dofus/network/messages'):
    fichiers=[os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f != '.DS_Store' and f != 'Ids.py']
    dossiers=[os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    for f in fichiers:
        o=open(f)
        r=o.read()
        o.close()
        first_occurence=r.find('protocolId:uint =')
        if first_occurence != -1: # means the value is found
            id_location=int(first_occurence + 18) # look 18 characters after (length of 'protocolId:uint = ') to know when the protocolID's value is declared
            end_first_occurence=int(r.find(';', id_location))
            indice=r[id_location: end_first_occurence]
            nom=os.path.basename(f)[:-3]
            dictIds[indice]=nom
            
    for d in dossiers:
        Ids(d)

def createIds(path='/Users/thomasaudevie/Desktop/projets-perso/kamarketplace/sources/scripts/com/ankamagames/dofus/network/messages'):
    ### crée le dictionnaire dictIds et le remplit
    global dictIds
    dictIds = dict()
    Ids(path)
    print('Terminé '+str(len(dictIds)))

import pickle

def dumpIds(path='./Ids'):
    ### crée le dictionnaire dictIds, le remplit et le sauvegarde
    createIds()
    f = open(path, 'wb')
    pickle.dump(dictIds, f)
    f.close()
    
def loadIds():
    ### charge le dictionnaire dictIds depuis le fichier
    f = open('./Ids', 'rb') #'/Users/louisabraham/Documents/Projet bot/Programme/Ids
    global dictIds
    dictIds = pickle.load(f)
    f.close()

In [95]:
dumpIds()

Terminé 1145


In [96]:
loadIds()

In [97]:
f = open('./Ids', 'rb')
global dictIds
dictIds = pickle.load(f)

In [99]:
dictIds['5432']

'GameRolePlayShowActorMessage'

# Deserialize